In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lid_driven_cavity import operators, states
import numpy as np

In [3]:
# Physics parameters
reynolds_numbers = [100, 400, 1000, 3200, 5000]

# Boundary conditions
u_top = (1, 0)
u_bot = (0, 0)
u_left = (0, 0)
u_right = (0, 0)

# Spatial parameters
Lx = 1.0
Ly = 1.0

In [4]:
# Physics parameter
Re = reynolds_numbers[-1]

# Spatial resolution
N = 3  # number of cells in each direction
h = 1.0 / N

# Temporal resolution
k = 1.0  # FIXME arbitrary for now, likely will need to decrease

# Setup

1. Assume $\phi^n$ and $u^n$ are known at all interior, boundary, and ghost points.

In [5]:
phi_n = states.State(np.zeros((N+2, N+2)), Nx=N+2, Ny=N+2)
u_n = states.State(np.zeros((N+2, N+1)), Nx=N+1, Ny=N+2)
v_n = states.State(np.zeros((N+1, N+2)), Nx=N+2, Ny=N+1)
print(phi_n)
print(u_n)
print(v_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


2. Assume $G(\phi^n)$ is known at all interior, boundary, and ghost points.

In [6]:
Gx_phi_n = operators.compute_gradient_x(phi_n)  # on phi grid, no left boundary
Gy_phi_n = operators.compute_gradient_y(phi_n)  # on phi grid, no bottom boundary
print(Gx_phi_n)
print(Gy_phi_n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


3. Assume $N(u^{n−1})$ is known at all interior points.

In [7]:
Nu_nm1 = states.State(np.zeros((N+2, N-1+2)), Nx=N-1+2, Ny=N+2)
Nv_nm1 = states.State(np.zeros((N-1+2, N+2)), Nx=N+2, Ny=N-1+2)
print(Nu_nm1)
print(Nv_nm1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


# Prediction step

4. Compute $\hat{u}^n$ at all interior points.

In [8]:
u_hat_n = operators.compute_interpolated_x(u_n)  # on phi grid, no boundary columns
v_hat_n = operators.compute_interpolated_y(v_n)  # on phi grid, no boundary rows
print(u_hat_n)
print(v_hat_n)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


5. Compute $N(u^n)$ at all interior points.

In [9]:
# Compute gradient of u
dudx = operators.compute_gradient_x_centered(u_n)  # on u grid, no boundary columns
dudy = operators.compute_gradient_y_centered(u_n)  # on u grid, no boundary rows
dudx = dudx.get_matrix()[1:-1, 1:-1]
dudy = dudy.get_matrix()[1:-1, 1:-1]
print(dudx)
print(dudy)

# Get interior points of u_n
u_n_interior = u_n.get_matrix()[1:-1, 1:-1]
print(u_n_interior)

# Interpolate v_hat in x direction
v_hat_interpolated = operators.compute_interpolated_x(v_hat_n)  # on u grid, no boundary rows
v_hat_interpolated = v_hat_interpolated.get_matrix()[:, 1:-1]
print(v_hat_interpolated)

# [u, v] dot grad(u)
Nu_interior = (-u_n_interior*dudx - v_hat_interpolated*dudy) / (2*h)
print(Nu_interior)
Nu = states.State(Nu_interior, Nx=Nu_interior.shape[1], Ny=Nu_interior.shape[0])
Nu.pad_boundaries()
print(Nu)

# # Compute gradient of v
# dvdx = v_n.matrix[:, 2:] - v_n.matrix[:, :-2]  # centered difference in x
# dvdx = dvdx[1:-1, :]  # remove ghost points (first and last rows)
# print(dvdx)
# dvdy = v_n.matrix[2:, :] - v_n.matrix[:-2, :]  # centered difference in y
# dvdy = dvdy[:, 1:-1]  # remove boundary points (first and last columns)
# print(dvdy)

# # Interpolate v_hat in x direction
# u_hat_interpolated = (u_hat_n.matrix[1:, :] + u_hat_n.matrix[:-1, :]) / 2
# u_hat_interpolated = u_hat_interpolated[1:-1, :]  # remove boundary points (first and last columns)
# print(u_hat_interpolated)

# # [u, v] dot grad(u)
# v_n_interior = v_n.matrix[1:-1, 1:-1]  # remove boundary and ghost points
# Nv = (-v_n_interior*dvdx - u_hat_interpolated*dvdy) / (2*h)
# print(Nv)

[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[-0. -0.]
 [-0. -0.]
 [-0. -0.]]
[[ 0.  0.  0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0. -0. -0.  0.]
 [ 0.  0.  0.  0.]]


6. Evaluate $g(x, t_{n+1})$ at all required locations on the domain boundary.

7. Solve for $u^*$ at all interior, boundary, and ghost points.

In [10]:
# Assemble Laplacian for u
u_ip1_j = u_n.matrix[:, 2:]
u_ip1_j = u_ip1_j[1:-1, :]
print(u_ip1_j)
u_i_jp1 = u_n.matrix[:-2, :]
u_i_jp1 = u_i_jp1[:, 1:-1]
print(u_i_jp1)
u_i_j = u_n.matrix[1:-1, 1:-1]
print(u_i_j)
u_im1_j = u_n.matrix[:, :-2]
u_im1_j = u_im1_j[1:-1, :]
print(u_im1_j)
u_i_jm1 = u_n.matrix[2:, :]
u_i_jm1 = u_i_jm1[:, 1:-1]
print(u_i_jm1)
Lu = (1.0*u_ip1_j + 1.0*u_i_jp1 - 4.0*u_i_j + 1.0*u_im1_j + 1.0*u_i_jm1) / h**2
print(Lu)

# Assemble Laplacian for v
v_ip1_j = v_n.matrix[:, 2:]
v_ip1_j = v_ip1_j[1:-1, :]
print(v_ip1_j)
v_i_jp1 = v_n.matrix[:-2, :]
v_i_jp1 = v_i_jp1[:, 1:-1]
print(v_i_jp1)
v_i_j = v_n.matrix[1:-1, 1:-1]
print(v_i_j)
v_im1_j = v_n.matrix[:, :-2]
v_im1_j = v_im1_j[1:-1, :]
print(v_im1_j)
v_i_jm1 = v_n.matrix[2:, :]
v_i_jm1 = v_i_jm1[:, 1:-1]
print(v_i_jm1)
Lv = (1.0*v_ip1_j + 1.0*v_i_jp1 - 4.0*v_i_j + 1.0*v_im1_j + 1.0*v_i_jm1) / h**2
print(Lv)

AttributeError: 'State' object has no attribute 'matrix'

In [ ]:
import scipy as sp
np.set_printoptions(linewidth=200, threshold=5000, precision=6)

# u_star solve
# Assemble RHS
b_interior = (k/2) * (3*Nu-Nu_nm1.matrix) + u_n.matrix[1:-1, 1:-1] + (k/2)*(1/Re)*Lu

# pad with zeros on boundaries
b = np.zeros((b_interior.shape[0]+2, b_interior.shape[1]+2))
b[1:-1, 1:-1] = b_interior

# convert to vector
b = states.get_vector(b)

# Assemble operator
laplacian, b = operators.assemble_laplacian_operator_u(b, Nx=N+1, Ny=N+2, h=h, k=k)
# print(laplacian.get_csr().toarray())

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

# Solve implicit step
u_star = sp.sparse.linalg.spsolve(A, b)
print(states.get_matrix(u_star, 4, 5))

# v_star solve
# Assemble RHS
b_interior = (k/2) * (3*Nv-Nv_nm1.matrix) + v_n.matrix[1:-1, 1:-1] + (k/2)*(1/Re)*Lv  # bookmarkj

# pad with zeros on boundaries
b = np.zeros((b_interior.shape[0]+2, b_interior.shape[1]+2))
b[1:-1, 1:-1] = b_interior

# convert to vector
b = states.get_vector(b)

# Assemble operator
laplacian, b = operators.assemble_laplacian_operator_v(b, Nx=N+2, Ny=N+1, h=h, k=k)
# print(laplacian.get_csr().toarray())

# Assemble linear system
A1 = sp.sparse.eye(laplacian.shape[0], dtype=float, format='csr')
A2 = (k/2)*(1/Re)*laplacian.get_csr()
A = A1 + A2

# Solve implicit step
v_star = sp.sparse.linalg.spsolve(A, b)
print(states.get_matrix(v_star, 5, 4))


# Poisson problem

8. Compute $D(u^*)$ at all interior points.

In [ ]:
u_star_matrix = states.get_matrix(u_star, 4, 5)
v_star_matrix = states.get_matrix(v_star, 5, 4)
print(u_star_matrix)
print(v_star_matrix)
dudx = u_star_matrix[:, 1:] - u_star_matrix[:, :-1]
dvdy = v_star_matrix[1:, :] - v_star_matrix[:-1, :]
print(dudx)
print(dvdy)
dudx_interior = dudx[1:-1, :]
dvdy_interior = dvdy[:, 1:-1]
print(dudx_interior)
print(dvdy_interior)
Dstar = (dudx_interior + dvdy_interior) / h
print(Dstar)

9. Solve for $\phi^{n+1}$ at all interior and ghost points.

In [ ]:
b_interior = Dstar / k

# pad with zeros
b = np.zeros((b_interior.shape[0]+2, b_interior.shape[1]+2))
b[1:-1, 1:-1] = b_interior

# convert to vector
b = states.get_vector(b)

# TODO add ones on main diagonal for state_ij = 0 entries!!!
Lphi, b = operators.assemble_laplacian_operator_phi(b, u_star_matrix, v_star_matrix, Nx=N+2, Ny=N+2, h=h, k=k)
print(Lphi.get_csr().toarray())

# Assemble linear system
A = Lphi.get_csr()
# A = (k/2)*(1/Re)*laplacian.get_csr()

phi_np1 = sp.sparse.linalg.spsolve(A, b)
print(phi_np1)

# Correction step

10. Compute $G(\phi^{n+1})$ at all interior, boundary, and ghost points.

In [ ]:
# G_phi_np1 = lid.gradient(phi_np1)

11. Correct $u^*$ to obtain $u^{n+1}$ at all interior, boundary, and ghost points.

In [ ]:
# u_np1 = lid.take_correction_step(u_star, G_phi_np1)

# Bookkeeping

12. Update counter from $n$ to $n + 1$ and time from $t_n$ to $t_{n+1}$.

In [ ]:
# n = n + 1
# tn = tn + k

13. Repeat from Step 1.